##  Program for reading, extracting and analyzing patents -- Version 6F_alloys


### Data Dictionary

####  df_patents (dataframe)  
 
 titles  -- patent titles 
 
 dates -- issue date of patents                            
 
 num_inventors -- number or inventors 
 
 num_pcs -- number of patent citations 
 
 num_npcs -- number of non-patent citations 
 
 num_rbs -- number of references by other patents 
 
 num_classes -- number of patent classifications 
 
 num_les -- number of legal events 
 
 num_patents -- number of patents  
 
 cum_rbs -- cumulative references (sum of num_rbs)

In [ ]:
# US Patents are reported by Google as html files.  Extract data from US Patents.  
# The requests library allows us to send a request to the API with the specific vaiables and parameters it takes
# The BeautifulSoup library allows us to parse the html formmated file we'll received after we make our request
 
import urllib2  
from bs4 import BeautifulSoup  
import numpy as np
import pandas as pd
import requests
from dateutil import parser
    
# read data into a DataFrame
df_patents = pd.read_csv('/Users/stevehansen/Desktop/alloy_patent_raw_data5300.csv')

# Concatenate http and patent_no to find the appropriate url for google patents.  

df_patents["url"] = df_patents["http"].map(str) + df_patents["patent_no"]
del df_patents["http"]
print (df_patents.head(8))

patent_url = df_patents["url"].tolist()
patent_url = str(patent_url) 
#print (len(patent_url))
 
patent_url = patent_url.replace(",","")
#print (len(patent_url))


# m is counter for http string, iterate over all https 
# n is counter for filling dataframe with consecutive numbers.   

m = 0
n = 0
titles = []
dates = []
num_inventors = []
num_pcs = []
num_npcs = [] 
num_rbs = []
num_classes = []
num_les = []
issue_dates = []
num_patents = []

# Each patent url is extracted by a string of 43 characters.  m <= 430 will extract 11 patents. (m = 73,229)
while m <= 208120:
    u1 = patent_url [m+2:m+42]
    print (u1)
    patent_info = requests.get(u1)  
    m = m + 43

# Save the text that was returned into a string
# Specify the URL patent information from Google using requests.get

    content_string = patent_info.content       
    print len(content_string)
    #print (content_string) 

# Cut the patent url from the string of patents 
# Parse with BeautifulSoup, store in variable `soup`

    soup = BeautifulSoup(content_string, 'html.parser')  
    name_box = soup.find('span', attrs={"class": "main-title"})  
    name = name_box.text.strip() # strip() is used to remove starting and trailing  

# Cut the 'Patent 1234567 - ' from the name and save the title
    title = name [19:]
    print (len(titles))
    print (title)  
    titles.append(title)
    df_patents.ix[n, 'title'] = title
    
    name_box = soup.find('tr', attrs={"class": 'patent-internal-links'}) 
    events = name_box.getText()
    print(events)

# Find the publication date of the patent. 
    name_box = soup.find('table', attrs={"class": "patent-bibdata"})  
    date = name_box.getText().replace("","")

#assignee = find "Publication date"  
    d1 = date.find("Publication date")
    d2 = date.find("Filing date")
    date = date[d1+16:d2]
    dt = parser.parse(date)
    print (dt)
    print ("Publication date = " + str(date))
    dates.append(date)
    issue_dates.append(dt)
    num_patents.append(n)

# Add values to dataframe.  
    df_patents.ix[n, 'issue_date'] = dt
    df_patents.ix[n, 'date'] = date
    df_patents.ix[n, 'num_patent'] = n
   
    
# Extract the inventors in this box. 
    name_box = soup.find('table', attrs={"class": "patent-bibdata"}) 
    inventors = name_box.getText()

    i1 = inventors.find("Inventor") 
    i2 = inventors.find("Original Assignee")

# Inventors are contained between i1 and i2 in the string, +9 eliminates 'Inventors' from showing
    inventors = inventors[i1+9:i2]
    num_inventor = inventors.count(",") + 1 
    print ("Number of inventors = " + str(num_inventor))
    num_inventors.append(num_inventor)
    df_patents.ix[n, 'num_inventor'] = num_inventor

    
# This section extracts the 5 key values from the Google patent page.  

    name_box = soup.find('tr', attrs={"class": 'patent-internal-links'}) 
    events = (name_box.getText().replace(",",""))
    #print(events)

    a = events.count('Patent Citations') 
    b = events.count('Non-Patent Citations')
    c = events.count('Referenced by')
    d = events.count('Classifications')
    e = events.count('Legal Events')
    f = events.count('Non')
    print (a,b,c,d,e,f)
    
    
# Remove 'Patent Citations' from event string. num_pc = number of patent citations.  

    if a > 1 and f >= 1: 
        a1 = events.find("(")
        pc = events[0:a1]
        a2 = events.find(")")
        num_pc = events[a1+1:a2]
        print ("Patent Citations = " + str(num_pc))
        num_pcs.append(num_pc)
    
        # slice off 'Patent Citations' from event string. 
        events = events [a2+2:] 
    
    elif a >= 1 and f < 1: 
        a1 = events.find("(")
        pc = events[0:a1]
        a2 = events.find(")")
        num_pc = events[a1+1:a2]
        print ("Patent Citations = " + str(num_pc))
        num_pcs.append(num_pc)
    
        # slice off 'Patent Citations' from event string. 
        events = events [a2+2:]   

    elif a <= 1 and f >= 1:
        num_pc = 0
        print ("Patent Citations = " + str(num_pc)) 
        num_pcs.append(num_pc)
        
    elif a <= 0: 
        num_pc = 0
        print ("Patent Citations = " + str(num_pc)) 
        num_pcs.append(num_pc)
        
    df_patents.ix[n, 'num_pc'] = num_pc
    
# Remove 'Non-Patent Citations' from event string.  num_npc = number of non-patent citations. 
    if b > 0: 
        b1 = events.find("(")
        npc = events[0:b1]
        b2 = events.find(")")
        num_npc = events[b1+1:b2]
        print ("Non-patent citations = " + str(num_npc))
        num_npcs.append(num_npc)
    
        # slice off 'Non-Patent Citations' from event string
        events = events [b2+2:]

    elif b <= 0: 
        num_npc = 0 
        print ("Non-patent Citations = " + str(num_npc))
        num_npcs.append(num_npc)
    df_patents.ix[n, 'num_npc'] = num_npc
    
# Remove 'Referenced by' citations from event string.  num_rb = number of referenced by citations.
    if c > 0: 
        c1 = events.find("(")
        rb = events[0:c1]
        c2 = events.find(")")
        num_rb = events[c1+1:c2]
        print ("Referenced by = " + str(num_rb))
        num_rbs.append(num_rb)
    
        # slice off 'Referenced by' from event string
        events = events [c2+2:]
        
    elif c <= 0: 
        num_rb = 0 
        print ("Referenced by = " + str(num_rb))
        num_rbs.append(num_rb)
    df_patents.ix[n, 'num_rb'] = num_rb

# Remove 'Classifications' citations.  num_classif = number of classifications
    if d > 0: 
        d1 = events.find("(")
        classif = events[0:d1]
        d2 = events.find(")")
        num_class = events[d1+1:d2]
        print ("Number classifications = " + str(num_class))
        num_classes.append(num_class)
        
        # slice off 'Classifications' from event string
        events = events [d2+2:]

    elif d <= 0: 
        num_class = 0 
        print ("Number classifications = " + str(num_class)) 
        num_classes.append(num_class)
    df_patents.ix[n, 'num_class'] = num_class

# Remove 'Legal Events' citations.
    if e > 0: 
        e1 = events.find("(")
        legal = events[0:e1]
        e2 = events.find(")")
        num_le = events[e1+1:e2]
        print ("Legal events =" + str(num_le))
        num_les.append(num_le)
    
        # slice off 'Legal Events' from event string
        events = events [e2+2:]

    elif e <= 0: 
        num_le = 0 
        print ("Legal Events = " + str (num_le))
        num_les.append(num_le)
    df_patents.ix[n, 'num_le'] = num_le
    n = n + 1
    
print (df_patents)


####  Save raw data to Output.csv  

In [ ]:
df_patents.to_csv('output_alloy_data_total.csv', encoding='utf-8')
len (df_patents) 

#### Import complete data set. 

In [1]:
import numpy as np
import pandas as pd

df_patents = pd.read_csv('/Users/stevehansen/Desktop/output_alloy_data_total.csv')
print len(df_patents)


10137


In [6]:
df_patents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10137 entries, 0 to 10136
Data columns (total 12 columns):
patent_no       10137 non-null object
url             10137 non-null object
title           10137 non-null object
issue_date      10137 non-null object
date            10137 non-null object
num_patent      10137 non-null int64
num_inventor    10137 non-null int64
num_pc          10137 non-null int64
num_npc         10137 non-null int64
num_rb          10137 non-null int64
num_class       10137 non-null int64
num_le          10137 non-null int64
dtypes: int64(7), object(5)
memory usage: 950.4+ KB


In [3]:
df_patents = df_patents.drop(['Unnamed: 0'], axis=1)
print (df_patents)   

      patent_no                                       url  \
0       9637179  https://www.google.com/patents/US9637179   
1       9636784  https://www.google.com/patents/US9636784   
2       9634324  https://www.google.com/patents/US9634324   
3       9634123  https://www.google.com/patents/US9634123   
4       9631879  https://www.google.com/patents/US9631879   
5       9631878  https://www.google.com/patents/US9631878   
6       9631268  https://www.google.com/patents/US9631268   
7       9631261  https://www.google.com/patents/US9631261   
8       9631260  https://www.google.com/patents/US9631260   
9       9631157  https://www.google.com/patents/US9631157   
10      9630834  https://www.google.com/patents/US9630834   
11      9630251  https://www.google.com/patents/US9630251   
12      9630161  https://www.google.com/patents/US9630161   
13      9629873  https://www.google.com/patents/US9629873   
14      9624568  https://www.google.com/patents/US9624568   
15      9624566  https:/

In [4]:
df_patents.describe()

,num_patent,num_inventor,num_pc,num_npc,num_rb,num_class,num_le
count,10137.000000,10137.000000,10137.000000,10137.000000,10137.000000,10137.000000,10137.000000
mean,5068.000000,2.919108,12.997139,3.212785,15.647036,16.174213,4.176186
std,2926.444173,1.818188,24.138194,8.621308,41.604616,10.058189,2.722414
min,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000,0.000000
25%,2534.000000,2.000000,5.000000,0.000000,2.000000,9.000000,2.000000
50%,5068.000000,3.000000,8.000000,0.000000,7.000000,14.000000,4.000000
75%,7602.000000,4.000000,15.000000,3.000000,18.000000,20.000000,6.000000
max,10136.000000,18.000000,981.000000,218.000000,2341.000000,116.000000,22.000000


### Turn string values into integers. 

In [5]:
#  Turn string values into integers. 

df_patents['num_inventor'] = df_patents['num_inventor'].astype(int)
df_patents['num_pc'] = df_patents['num_pc'].astype(int)
df_patents['num_npc'] = df_patents['num_npc'].astype(int)
df_patents['num_rb'] = df_patents['num_rb'].astype(int)
df_patents['num_class'] = df_patents['num_class'].astype(int)
df_patents['num_le'] = df_patents['num_le'].astype(int)
df_patents['num_patent'] = df_patents['num_patent'].astype(int)


### Reverse order of the data in the dataframe.  

In [7]:
df_patents = df_patents.iloc[::-1]

In [8]:
print (df_patents)

      patent_no                                       url  \
10136   3930575  https://www.google.com/patents/US3930575   
10135   3930848  https://www.google.com/patents/US3930848   
10134   3930895  https://www.google.com/patents/US3930895   
10133   3930899  https://www.google.com/patents/US3930899   
10132   3930904  https://www.google.com/patents/US3930904   
10131   3930962  https://www.google.com/patents/US3930962   
10130   3930964  https://www.google.com/patents/US3930964   
10129   3930965  https://www.google.com/patents/US3930965   
10128   3930966  https://www.google.com/patents/US3930966   
10127   3933473  https://www.google.com/patents/US3933473   
10126   3933481  https://www.google.com/patents/US3933481   
10125   3933483  https://www.google.com/patents/US3933483   
10124   3933484  https://www.google.com/patents/US3933484   
10123   3933531  https://www.google.com/patents/US3933531   
10122   3933961  https://www.google.com/patents/US3933961   
10121   3935007  https:/